In [112]:
import psycopg2
import os
import sys

def getOpenConnection(user='postgres', password='1234', dbname='ddsassignment3'):
    return psycopg2.connect("dbname='" + dbname + "' user='" + user + "' host='localhost' password='" + password + "'")

In [113]:
con = getOpenConnection()

In [114]:
#Assignment3.ParallelSort('ratings', 'Rating', 'parallelSortOutputTable', con);

In [116]:
cur = con.cursor()
query = "select  min({0}), max({0}) from {1} ".format('Rating', 'ratings')
cur.execute(query)
val = cur.fetchone()
print val

(1.0, 5.0)


In [107]:
cur = con.cursor()
query = "select  max({0}) from {1} ".format('Rating', 'ratings')
cur.execute(query)
max_val = cur.fetchone()[0]

In [108]:
interval = (max_val - min_val)/5 

In [109]:
interval

0.8

In [117]:
def parallel(InputTable, rangetable, SortingColumnName, lowerbound, upperbound, openconnection):
    cur = openconnection.cursor()
    if rangetable == partition + str(0):
        query = 'Insert into {0} select * from {1} where {2} >= {3} and {2} <= {4} order by {2} asc'.format(rangetable, InputTable, SortingColumnName, lowerbound, upperbound)
    else:
        query = 'Insert into {0} select * from {1} where {2} >= {3} and {2} <= {4} order by {2} asc'.format(rangetable, InputTable, SortingColumnName, lowerbound, upperbound)
    cur.execute(query)


import threading
def copytable(sourcetable,destinationtable,cur):
    query = "CREATE TABLE {0} AS SELECT * FROM {1} WHERE 1=2".format(destinationtable,sourcetable)
    cur.execute(query);

partition = 'rangepartition'
start = min_val
end = max_val
index = 0
length = (max_val - min_val) * 1.0 / 5
threads = range(5)
while start < end:
    tablename = partition + str(index)
    interval = start + 0.8
    print tablename, start, interval
    copytable('ratings', tablename , cur)
    threads[index] = threading.Thread(target = parallel, args = ('ratings', tablename, 'Rating', start, end, con))
    threads[index].start()
    start = interval
    index += 1
    
for i in range(5):
    threads[i].join()

print "done"
    
for i in range(TOTAL_THREADS):
    tablename = partition + repr(i)
    query = "INSERT INTO {0} SELECT * FROM {1}".format( 'OutputTable', tablename)
    cur.execute(query)
        
for i in range(TOTAL_THREADS):
    tablename = partition + repr(i)
    cur.execute('DROP TABLE IF EXISTS {0} CASCADE'.format(tablename))
    
cur.close()
con.commit()
con.close()

rangepartition0 1.0 1.8


TransactionRollbackError: deadlock detected
DETAIL:  Process 9224 waits for AccessShareLock on object 44025 of class 2615 of database 43950; blocked by process 8983.
Process 8983 waits for AccessExclusiveLock on relation 44026 of database 43950; blocked by process 9224.
HINT:  See server log for query details.


[0, 1, 2, 3, 4]